In [6]:
!/bin/bash ./build_and_push.sh accessbank-catboost-ctr

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  55.04MB
Step 1/11 : FROM tensorflow/tensorflow:2.0.0-py3
 ---> 90f5cb97b18f
Step 2/11 : MAINTAINER Akinwande Komolafe<akomolafe@terragonltd.com>
 ---> Using cache
 ---> fb67125fcbc2
Step 3/11 : RUN apt-get update && apt-get install -y --no-install-recommends nginx curl
 ---> Using cache
 ---> 41c3c3b7c0ad
Step 4/11 : COPY requirements.txt /opt/program/requirements.txt
 ---> Using cache
 ---> d9d8e1db2f9b
Step 5/11 : RUN pip install -r /opt/program/requirements.txt
 ---> Using cache
 ---> 7a7c3b57b651
Step 6/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> bf67f31a5ed9
Step 7/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 

In [26]:
from sagemaker import get_execution_role

role = get_execution_role()

# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./local_setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [2]:
pwd

'/home/ec2-user/SageMaker/Accessbank_CTR_1/Catboost-sagemaker'

In [ ]:
import boto3,re
from sagemaker.estimator import Estimator


instance_type = 'ml.m4.2xlarge'
output = 's3://datateam-ml/Adrenaline_November_CTR/catboost_access_ctr/' 
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name='101063123548.dkr.ecr.eu-west-1.amazonaws.com/accessbank-catboost-ctr',
                     output_path = output)

estimator.fit('s3://datateam-ml/Adrenaline-November-CTR/data/train')

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-11-23 00:49:32 Starting - Starting the training job...
2020-11-23 00:49:34 Starting - Launching requested ML instances..................
2020-11-23 00:52:51 Starting - Preparing the instances for training...
2020-11-23 00:53:29 Downloading - Downloading input data...
2020-11-23 00:53:46 Training - Downloading the training image......
2020-11-23 00:54:45 Training - Training image download completed. Training in progress.True. /opt/ml/input/config/hyperparameters.json doesnt exist

Starting the training.
['/opt/ml/input/data/training/Train_sms.csv', '/opt/ml/input/data/training/Train_click.csv']
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
 Features 

In [1]:
pwd

'/home/ec2-user/SageMaker/Accessbank_CTR_1/Catboost-sagemaker'

In [ ]:
## SageMaker BatchTrandform Job
import sagemaker
import json
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sagemaker.Session()

input_location = 's3://datateam-ml/flat_demography/year=2020/month=03'
output_location = 's3://datateam-ml/Adrenaline_November_CTR/new_predictions'

# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(
        base_transform_job_name='catboost-accessbank-ctr',
        model_name='catboost-access-ctr',
        instance_count=1,
        instance_type='ml.m4.2xlarge',
        output_path=output_location,
        accept='text/csv',
        max_payload=100,
        assemble_with = 'Line')
    
# To start a transform job:
transformer.transform(data=input_location, content_type='text/csv', job_name='catboost-accessbank-ctr-7',\
                      data_type='S3Prefix', split_type='Line', input_filter="$", \
                      join_source= "None", output_filter="$")

# Then wait until transform job is completed
transformer.wait()

..........................2020-11-24T10:22:43.469:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=100, BatchStrategy=MULTI_RECORD
Starting the inference server with 8 workers.
2020/11/24 10:22:41 [crit] 10#10: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [24/Nov/2020:10:22:41 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2020/11/24 10:22:41 [crit] 10#10: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [24/Nov/2020:10:22:41 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
[2020-11-24 10:22:41 +0000] [9] 